In [65]:
import os
print(os.getcwd())

C:\Users\sawya\Documents


In [66]:
import duckdb

tables = con.exec
con = duckdb.connect(database='mimic.db')

# Get tablesute("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'main'
""").fetchall()

if not tables:
    print("No tables found in the 'main' schema")
else:
    for (table,) in tables:
        print(f"Attempting to drop: {table}")
        try:
            con.execute(f"DROP TABLE IF EXISTS {table}")
            print(f"Successfully dropped {table}")
        except Exception as e:
            print(f"Error dropping {table}: {str(e)}")

Attempting to drop: caregiver
Successfully dropped caregiver
Attempting to drop: chartevents
Successfully dropped chartevents
Attempting to drop: datetimeevents
Successfully dropped datetimeevents
Attempting to drop: d_items
Successfully dropped d_items
Attempting to drop: icustays
Successfully dropped icustays
Attempting to drop: ingredientevents
Successfully dropped ingredientevents
Attempting to drop: inputevents
Successfully dropped inputevents
Attempting to drop: outputevents
Successfully dropped outputevents
Attempting to drop: procedureevents
Successfully dropped procedureevents


In [67]:
#import tables from icu
import duckdb
import os

# Set your folder path
data_folder = r"C:\Users\sawya\OneDrive\Documents\SQL\mimic-iv-clinical-database-demo-2.2\mimic-iv-clinical-database-demo-2.2\icu"
# Create DuckDB connection (in-memory or persistent)
con = duckdb.connect(database='mimic.db')  

# Loop through all .csv.gz files and register as tables
for file in os.listdir(data_folder):
    if file.endswith(".csv.gz"):
        file_path = os.path.join(data_folder, file)
        table_name = file.replace(".csv.gz", "")

        print(f"Registering table: {table_name}")

        con.execute(f"""
                    CREATE TABLE {table_name} AS
                     SELECT * FROM read_csv_auto('{file_path}', ignore_errors=true) """)

Registering table: caregiver
Registering table: chartevents


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Registering table: datetimeevents
Registering table: d_items
Registering table: icustays
Registering table: ingredientevents
Registering table: inputevents
Registering table: outputevents
Registering table: procedureevents


In [68]:
#import tables from hosp
import duckdb
import os

# Set your folder path
data_folder = r"C:\Users\sawya\OneDrive\Documents\SQL\mimic-iv-clinical-database-demo-2.2\mimic-iv-clinical-database-demo-2.2\hosp"

# Create DuckDB connection (in-memory or persistent)
con = duckdb.connect(database='mimic.db')  

# Loop through all .csv.gz files and register as tables
for file in os.listdir(data_folder):
    if file.endswith(".csv.gz"):
        file_path = os.path.join(data_folder, file)
        table_name = file.replace(".csv.gz", "")

        print(f"Registering table: {table_name}")

        con.execute(f"""
                    CREATE TABLE {table_name} AS
                     SELECT * FROM read_csv_auto('{file_path}', ignore_errors=true) """)

Registering table: admissions
Registering table: diagnoses_icd
Registering table: drgcodes
Registering table: d_hcpcs
Registering table: d_icd_diagnoses
Registering table: d_icd_procedures
Registering table: d_labitems
Registering table: emar
Registering table: emar_detail
Registering table: hcpcsevents
Registering table: labevents
Registering table: microbiologyevents
Registering table: omr
Registering table: patients
Registering table: pharmacy
Registering table: poe
Registering table: poe_detail
Registering table: prescriptions
Registering table: procedures_icd
Registering table: provider
Registering table: services
Registering table: transfers


 <h1 style="font-size: 20px; font-weight: 800; color: #333333; margin-bottom: 20px; border: none !important; background: none !important;">
    Patient Demographics
</h1>

In [ ]:
#Count patients by gender and average age at first admission.


import pandas as pd
import duckdb

con = duckdb.connect(database='mimic.db')
query = """

WITH first_time AS (

SELECT 
        p.subject_id,
        p.gender,
        p.anchor_age,
        p.anchor_year,
        MIN(a.admittime) AS  first_admittime
FROM
    patients p
    
JOIN 
    admissions a ON p.subject_id = a.subject_id
    
GROUP BY 
    p.subject_id, p.gender, p.anchor_age, p.anchor_year
)

SELECT 
        gender,
        COUNT(subject_id) AS patients_counts,
        ROUND(AVG(anchor_age + (YEAR(first_admittime)-anchor_year)),1) AS average_age
FROM 
    first_time
GROUP BY 
    gender
ORDER BY 
    patients_counts;
"""

result_df = con.execute(query).df()
result_df

<h1 style="font-size: 20px; font-weight: 700; color: #333333; margin-bottom: 10px; border: none !important; background: none !important;">
    Top lab tests ordered per department
</h1>

In [9]:
# using services and labevents


import pandas as pd
import duckdb

con = duckdb.connect(database = "mimic.db")

query = """

SELECT

le.subject_id, 
s.curr_service, 
li.label AS lab_test_name, 
COUNT(*) AS test_counts

FROM labevents le
JOIN services s on le.subject_id = s.subject_id
JOIN d_labitems li on le.itemid = li.itemid
WHERE s.curr_service IS NOT NULL
GROUP BY le.subject_id,li.label,s.curr_service
ORDER BY s.curr_service, test_counts DESC;
"""
result_df = con.execute(query).df()
result_df



,subject_id,curr_service,lab_test_name,test_counts
0,10014354,CMED,L,895
1,10014354,CMED,H,895
2,10014354,CMED,I,895
3,10014354,CMED,Sodium,760
4,10014354,CMED,Potassium,760
...,...,...,...,...
15835,10005909,VSURG,Ferritin,1
15836,10007058,VSURG,Thyroid Stimulating Hormone,1
15837,10022281,VSURG,Monocytes,1
15838,10005909,VSURG,Thyroid Stimulating Hormone,1


<h1 style="font-size: 20px; font-weight: 700; color: #333333; margin-bottom: 10px; border: none !important; background: none !important;">
    Identify patients with early high-risk vitals in first 6 hours of ICU
</h1>

In [14]:
#(e.g., hypotension)

import duckdb
import pandas as pd


con = duckdb.connect(database='mimic.db')

query = """

WITH high_risk_vitals AS(

SELECT 
ce.subject_id,
ce.hadm_id,
ce.valuenum,
ce.itemid,
ce.charttime,
i.intime,
li.label
FROM chartevents ce
JOIN icustays i on ce.subject_id = i.subject_id
JOIN d_items li ON ce.itemid = li.itemid
WHERE ce.charttime BETWEEN i.intime AND DATE_ADD(i.intime, INTERVAL 6 HOUR)
AND li.label IN ('Heart Rate', 'Systolic Blood Pressure')
),

flags AS(

SELECT *,
CASE 
WHEN label = 'Systolic Blood Pressure' AND valuenum < 90 THEN 1
WHEN label = 'Heart Rate' AND valuenum > 120 THEN 1
ELSE 0
END AS risk_flags
FROM high_risk_vitals
)

SELECT 

hadm_id, subject_id,
COUNT(*) AS total_readings,
COUNT(CASE WHEN risk_flags = 1 THEN 1 END) AS high_risk_readings,
ROUND(100.0*COUNT(CASE WHEN risk_flags = 1 THEN 1 END)/COUNT(*),2) AS risk_percentage
FROM flags
GROUP BY hadm_id, subject_id
ORDER BY risk_percentage DESC
LIMIT 10;


"""

result_df = con.execute(query).df()
result_df




,hadm_id,subject_id,total_readings,high_risk_readings,risk_percentage
0,27993466,10015272,4,4,100.00
1,27617929,10037975,8,8,100.00
2,25129047,10038933,8,6,75.00
3,20291550,10008454,7,4,57.14
4,29600294,10014354,7,3,42.86
5,24181354,10004235,8,3,37.50
6,21607814,10023117,9,3,33.33
7,20214994,10003400,24,7,29.17
8,26275841,10027445,7,2,28.57
9,28477357,10007795,7,2,28.57


<h1 style="font-size: 20px; font-weight: 700; color: #333333; margin-bottom: 10px; border: none !important; background: none !important;">
   Admission Statistics
</h1>

In [27]:
#Calculate average length of stay by admission type (ELECTIVE/URGENT).

import pandas as pd
import duckdb

con = duckdb.connect(database='mimic.db')
query = """
SELECT 
    admission_type,
    ROUND(AVG(EXTRACT(EPOCH FROM(dischtime - admittime))/86400),2) AS avg_length_of_stay
FROM 
    admissions
WHERE 
    dischtime IS NOT NULL 
AND 
    admittime IS NOT NULL 
AND 
    dischtime > admittime
GROUP BY 
    admission_type
ORDER BY 
    avg_length_of_stay DESC;

"""

result_df = con.execute(query).df()
result_df

,admission_type,avg_length_of_stay
0,URGENT,9.85
1,DIRECT EMER.,9.39
2,ELECTIVE,8.19
3,OBSERVATION ADMIT,8.15
4,EW EMER.,7.28
5,SURGICAL SAME DAY ADMISSION,5.70
6,DIRECT OBSERVATION,1.45
7,AMBULATORY OBSERVATION,1.02
8,EU OBSERVATION,0.92


<h1 style="font-size: 20px; font-weight: 700; color: #333333; margin-bottom: 10px; border: none !important; background: none !important;">
    Common Diagnoses across all patients
</h1>

In [21]:
#List the top 10 most frequent ICD-9 diagnosis codes and their descriptions

import pandas as pd
import duckdb

con = duckdb.connect(database='mimic.db')
query = """

SELECT 
    d.long_title,
    d.icd_code,
    COUNT(d.icd_code) AS diagnosis_count,
    DENSE_RANK() OVER (ORDER BY COUNT(d.icd_code) DESC) AS rank
FROM 
    d_icd_diagnoses d
JOIN 
    diagnoses_icd di ON d.icd_code = di.icd_code AND d.icd_version = di.icd_version
WHERE 
    d.icd_version = 9
GROUP BY
    d.long_title, d.icd_code
   

"""
result_df = con.execute(query).df()
result_df

,long_title,icd_code,diagnosis_count,rank
0,Unspecified essential hypertension,4019,68,1
1,Other and unspecified hyperlipidemia,2724,55,2
2,Atrial fibrillation,42731,34,3
3,Diabetes mellitus without mention of complicat...,25000,33,4
4,Personal history of tobacco use,V1582,31,5
...,...,...,...,...
735,Intravenous anesthetics causing adverse effect...,E9383,1,31
736,"Accidental cut, puncture, perforation or hemor...",E8708,1,31
737,Rhabdomyolysis,72888,1,31
738,Other specified peritonitis,56789,1,31


In [8]:
# Addition --- Top 10 Diagnoses by Patient Count

import duckdb
import pandas as pd

con = duckdb.connect(database = "mimic.db")

query = """
SELECT d.icd_code, dd.long_title, COUNT(DISTINCT d.subject_id) AS patient_count
FROM diagnoses_icd d
JOIN d_icd_diagnoses dd ON d.icd_code = dd.icd_code
GROUP BY d.icd_code, dd.long_title
ORDER BY patient_count DESC
LIMIT 10;
"""
result_df = con.execute(query).df()
result_df


,icd_code,long_title,patient_count
0,4019,Unspecified essential hypertension,41
1,2724,Other and unspecified hyperlipidemia,34
2,E785,"Hyperlipidemia, unspecified",21
3,25000,Diabetes mellitus without mention of complicat...,21
4,V1582,Personal history of tobacco use,20
5,41401,Coronary atherosclerosis of native coronary ar...,19
6,42731,Atrial fibrillation,18
7,3051,Tobacco use disorder,18
8,2859,"Anemia, unspecified",18
9,I10,Essential (primary) hypertension,17


<h1 style="font-size: 20px; font-weight: 700; color: #333333; margin-bottom: 10px; border: none !important; background: none !important;">
    Medication Prescriptions
</h1>

In [90]:
#Find the 5 most prescribed drugs for patients over 65.

import pandas as pd
import duckdb

con = duckdb.connect(database='mimic.db')
query = """

SELECT 
    COUNT(p.drug) AS drugs_counts,
    p.drug,
    DENSE_RANK() OVER (ORDER BY COUNT(p.drug) DESC) AS rank
FROM 
    prescriptions p
JOIN 
    patients pp on p.subject_id = pp.subject_id
WHERE 
    pp.anchor_age > 65
GROUP BY 
    p.drug
ORDER BY 
    drugs_counts DESC
LIMIT 5;
"""
result_df = con.execute(query).df()
result_df


,drugs_counts,drug,rank
0,293,Insulin,1
1,287,0.9% Sodium Chloride,2
2,256,Potassium Chloride,3
3,208,Sodium Chloride 0.9% Flush,4
4,192,Metoprolol Tartrate,5


<h1 style="font-size: 20px; font-weight: 700; color: #333333; margin-bottom: 10px; border: none !important; background: none !important;">
    Average length of stay between first admissions and readmissions
</h1>

In [34]:
#Compare within 30 days

import pandas as pd
import duckdb

con = duckdb.connect(database='mimic.db')
query = """

WITH admission_sequence AS (
    SELECT 
        subject_id,
        hadm_id,
        admittime,
        dischtime,
        EXTRACT(EPOCH FROM (dischtime-admittime))/86400 As los_days,
        ROW_NUMBER() OVER (PARTITION BY subject_id ORDER BY admittime) AS admission_num,
        LAG(dischtime) OVER (PARTITION BY subject_id ORDER BY admittime) AS prev_dischtime
        FROM admissions
        WHERE admittime IS NOT NULL 
        AND dischtime IS NOT NULL
),

categorized_admissions AS (
    SELECT
        *,
        CASE WHEN admission_num = 1 THEN 'First_admissions'
             WHEN  EXTRACT(EPOCH FROM (admittime-prev_dischtime))/86400 <=30 
             AND prev_dischtime IS NOT NULL THEN 'Readmission ≤30 days'
             ELSE 'other days'
             END AS admission_type
FROM  admission_sequence 
)

SELECT 

admission_type, 
ROUND(AVG(los_days),2) AS avg_los_days,
COUNT(*) AS admission_count
FROM categorized_admissions
WHERE admission_type IN ('First_admissions','Readmission ≤30 days')
GROUP BY admission_type
ORDER BY admission_count DESC;
  

"""
result_df = con.execute(query).df()
result_df        
        

,admission_type,avg_los_days,admission_count
0,First_admissions,8.05,100
1,Readmission ≤30 days,7.83,53


<h1 style="font-size: 20px; font-weight: 700; color: #333333; margin-bottom: 10px; border: none !important; background: none !important;">
    Most common medications given in the first 24 hours of ICU stay
</h1>

In [36]:
 

import pandas as pd
import duckdb

con = duckdb.connect(database='mimic.db')
query = """

   
    SELECT 
    d.label AS medication,
    COUNT(*) AS times_administered
FROM 
    inputevents i
JOIN 
    d_items d ON i.itemid = d.itemid
JOIN 
    icustays icu ON i.stay_id = icu.stay_id
WHERE 
    i.starttime BETWEEN icu.intime AND icu.intime + INTERVAL 24 HOUR
GROUP BY 
    d.label
ORDER BY 
    times_administered DESC
LIMIT 20;
"""
result_df = con.execute(query).df()
result_df

,medication,times_administered
0,NaCl 0.9%,989
1,Dextrose 5%,776
2,Solution,424
3,Propofol,312
4,Norepinephrine,302
5,Insulin - Regular,256
6,PO Intake,230
7,Phenylephrine,226
8,LR,157
9,Fentanyl,140


<h1 style="font-size: 20px; font-weight: 700; color: #333333; margin-bottom: 10px; border: none !important; background: none !important;">
    The percentage of abnormal potassium results 
</h1>

In [16]:
 #Calculate the percentage of abnormal potassium results (<3.5 or >5.0 mEq/L) by first_care unit(ICU type)"

import pandas as pd
import duckdb

con = duckdb.connect(database='mimic.db')
query = """

SELECT 
    i.first_careunit AS icu_type,
    d.label,
    COUNT(*) AS total_labs,
    SUM(CASE WHEN l.valuenum < 3.5 OR l.valuenum > 5.0 THEN 1 ELSE 0 END) AS  abnormal_results,
    ROUND(100.0*SUM(CASE WHEN l.valuenum < 3.5 OR l.valuenum > 5.0 THEN 1 ELSE 0 END)/ COUNT(*),2) AS pct_abnormal
FROM labevents l
JOIN icustays i ON l.hadm_id = i.hadm_id
JOIN d_labitems d ON l.itemid = d.itemid
WHERE LOWER (d.label) like '%potassium%'
GROUP BY i.first_careunit,d.label
ORDER BY pct_abnormal DESC;

"""
result_df = con.execute(query).df()
result_df


,icu_type,label,total_labs,abnormal_results,pct_abnormal
0,Medical Intensive Care Unit (MICU),"Potassium, Urine",10,10.0,100.00
1,Medical/Surgical Intensive Care Unit (MICU/SICU),"Potassium, Urine",8,8.0,100.00
2,Surgical Intensive Care Unit (SICU),"Potassium, Urine",4,4.0,100.00
3,Neuro Surgical Intensive Care Unit (Neuro SICU),"Potassium, Urine",1,1.0,100.00
4,Neuro Intermediate,Potassium,1,1.0,100.00
5,Trauma SICU (TSICU),"Potassium, Urine",7,7.0,100.00
6,Coronary Care Unit (CCU),"Potassium, Urine",4,4.0,100.00
7,Neuro Surgical Intensive Care Unit (Neuro SICU),"Potassium, Whole Blood",1,1.0,100.00
8,Coronary Care Unit (CCU),"Potassium, Whole Blood",6,4.0,66.67
9,Medical Intensive Care Unit (MICU),"Potassium, Whole Blood",19,10.0,52.63


<h1 style="font-size: 20px; font-weight: 700; color: #333333; margin-bottom: 10px; border: none !important; background: none !important;">
    In-hospital mortality rate per top 5 sepsis-related diagnoses
</h1>

In [11]:
# (10) In-hospital mortality rate per top 5 sepsis-related diagnoses

import duckdb
import pandas as pd


con = duckdb.connect(database='mimic.db')

query = """
WITH sepsis_codes AS (
  SELECT icd_code
  FROM d_icd_diagnoses
  WHERE LOWER(long_title) LIKE '%sepsis%'
),
sepsis_cases AS (
  SELECT d.subject_id, d.hadm_id, d.icd_code, d.icd_version
  FROM diagnoses_icd d
  JOIN sepsis_codes s ON d.icd_code = s.icd_code
),
mortality_flags AS (
  SELECT hadm_id, 
        CASE WHEN deathtime IS NOT NULL AND dischtime IS NOT NULL AND deathtime <= dischtime THEN 1 ELSE 0 END AS died
  FROM admissions
)
SELECT 
  d.icd_code,
  d.icd_version,
  COUNT(*) AS total_cases,
  SUM(m.died) AS deaths,
  ROUND(100.0 * SUM(m.died) / COUNT(*), 2) AS mortality_rate_pct
FROM sepsis_cases d
JOIN mortality_flags m ON d.hadm_id = m.hadm_id
GROUP BY d.icd_code, d.icd_version
ORDER BY mortality_rate_pct DESC
LIMIT 5;


"""

result_df = con.execute(query).df()
result_df

,icd_code,icd_version,total_cases,deaths,mortality_rate_pct
0,R6520,10,3,1.0,33.33
1,A419,10,7,2.0,28.57
2,R6521,10,7,1.0,14.29
3,99592,9,8,1.0,12.50
4,A4151,10,1,0.0,0.00
